In [33]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from scripts.all_imports import *
from scripts.utils import  CONFIG, FILE_PATHS, PATH_ROOT, INPUT_PATH, OUTPUT_PATH

print("Configuration:", CONFIG)
print("Path:", FILE_PATHS)
print("Source:", PATH_ROOT)
print("Input:",INPUT_PATH)
print("Output:",OUTPUT_PATH)
print("File",CONFIG["files"]["file_data_pubmed"])


Configuration: {'api_key': '09921aadccbafb8e5913ae1c99e2fafbe00a', 'api_email': 'leandrorodrigues.s@gmail.com', 'config': {'search_type': 1, 'time_sleep': '0.1', 'file_save_periodically': 10, 'save_xml': True, 'search': 0.1, 'search_article_one': True}, 'directories': {'output': './data/processed', 'xml': './data/processed/xml', 'input': './data/input', 'logs': './logs', 'type_files': './data/processed/files_type'}, 'files': {'file_data_error': './metadata/NotProcessed.csv', 'file_data_input': 'References_MJFF_15122024.csv', 'file_parsing_pubmed': './metadata/PubMed_Metadata.csv', 'file_parsing_bibliometrix': './metadata/Bibliometrix_Metadata.csv', 'file_data_pubmed': './files_type/PubMed_format.txt', 'file_bibliometrix': './files_type/Bibliometrix_format.xlsx', 'file_vsviewer': 'VosViewer.ris', 'file_bibtex': 'bibtex.bib', 'file_web_of_science': 'web_of_science.txt'}, 'logging': {'log_file': './logs/execution.log', 'level': 'INFO', 'format': '%(asctime)s - %(levelname)s - %(message)s'

In [34]:
logging.info("Application started.")
print(f" paths: Input -> {INPUT_PATH} \n output -> {OUTPUT_PATH}")
df_pubmed = pd.DataFrame()
    
logging.info(f"Output file paths: {FILE_PATHS}")
logging.info(f"Output config: {CONFIG}")

 paths: Input -> c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\input 
 output -> c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed


In [35]:
# Define the file path for the input data. If the key "file_data_input" exists in the configuration,
# its value will be used. Otherwise, the default file name "InputSearch.csv" is used.
# Usage of the function
FILE_INPUT_SEARCH = os.path.join(INPUT_PATH, CONFIG["files"].get("file_data_input", "InputSearchDoi.csv"))
print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_search = load_csv_to_dataframe(FILE_INPUT_SEARCH, ';')

Search file: c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\input\References_MJFF_15122024.csv


In [36]:
FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_pubmed", "PubMed_Metadata.csv"))
FILE_INPUT_SEARCH = os.path.normpath(FILE_INPUT_SEARCH)

print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_pubmed = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')

pd.set_option('display.max_columns', None)
df_pubmed.head(2)


Search file: c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed\metadata\PubMed_Metadata.csv


,PMID,Owner,MedlineCitation.Status,DataBankName,PublicationStatus,DateCompleted,DateRevised,ArticleDate,ELocationDOI,LocationPII,ArticleIdListPII,ArticleIdListDOI,ISSN,ISSNLinking,JournalIssue.Volume,JournalIssue.Issue,PubDate.Year,PublicationSeason,ArticleTitle,Abstract.AbstractText,CopyrightInformation,StartPage,MedlinePgn,AccessionNumber,PubMedPubDateReceived,PubMedPubDateAccepted,PubMedPubDateMedline,PubMedPubDatePubmed,PubMedPubDateEntrez,Country,MedlineTA,Title,NlmUniqueID,CitationSubset,MeshHeadingList,Keyword,KeywordList,ArticleIdListPMC,Language,PublicationType,AuthorListAuthor,AuthorListDTL,CoiStatement
0,36444971,NLM,MEDLINE,Unknown,ppublish,20230125,20230222,20221129,10.1002/mds.29262,Unknown,Unknown,10.1002/mds.29262,1531-8257,0885-3185,38,1,2023,Unknown,Practically Defined Off-State Dyskinesia Follo...,BACKGROUND: We recently showed that by employi...,© 2022 The Authors. Movement Disorders publish...,104,104-112,Unknown,2022/05/15 00:00,2022/10/07 00:00,2023/01/21 06:00,2022/11/30 06:00,2022/11/29 07:43,United States,Mov Disord,Movement disorders : official journal of the M...,8610688,IM,Humans; Dopamine; *Dyskinesias; Glial Cell Lin...,Unknown,NaN,Unknown,eng,"Journal Article; Research Support, Non-U.S. Gov't","Lloyd, Katherine| Lawton, Michael| Whone, Alan","FAU:Lloyd, Katherine|AU:Lloyd K|AUID:0000-0001...",Unknown
1,36503211,NLM,PubMed-not-MEDLINE,Unknown,ppublish,NaN,20230106,20221130,10.5213/inj.2222206.103,Unknown,inj.2222206.103,10.5213/inj.2222206.103,Unknown,2093-4777,26,Suppl 2,2022,Unknown,Regulation of Αlpha-Synuclein Gene (SNCA) by E...,PURPOSE: Deregulation of SNCA encoding α-synuc...,Unknown,S85,S85-93,Unknown,2022/09/15 00:00,2022/10/12 00:00,2022/12/13 06:01,2022/12/13 06:00,2022/12/12 09:02,Korea (South),Int Neurourol J,International neurourology journal,101534513,Unknown,NaN,NOTNLM,Parkinson disease; Repressor; TET1; Αlpha-synu...,PMC9767688,eng,Journal Article,"Guhathakurta, Subhrangshu| Song, Min Kyung| Ba...","FAU:Guhathakurta, Subhrangshu|AU:Guhathakurta ...",


In [37]:
FILE_PARSING_PUBMED = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_data_pubmed", "PubMed_Metadata.txt"))
FILE_PARSING_PUBMED = os.path.normpath(FILE_PARSING_PUBMED)

try:
    if os.path.isfile(FILE_PARSING_PUBMED):
        os.remove(FILE_PARSING_PUBMED)
        print(f"File {FILE_PARSING_PUBMED} has been successfully deleted.")
    else:
        print(f"File {FILE_PARSING_PUBMED} does not exist or is not a valid file.")

    map_to_pubmed_format(df_pubmed, FILE_PARSING_PUBMED)

    print(f"Saving Pubmed file: {FILE_PARSING_PUBMED}")
    logging.info(f"Saving Pubmed file: {FILE_PARSING_PUBMED}") 
    
except Exception as e:
    print(f"An error occurred while trying to delete the file: {e}")

File c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed\files_type\PubMed_format.txt has been successfully deleted.
An error occurred mapping.py: 'float' object has no attribute 'split'
Saving Pubmed file: c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed\files_type\PubMed_format.txt


In [6]:
FILE_INPUT_SEARCH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_parsing_bibliometrix", "Bibliometrix_Metadata.csv"))
FILE_INPUT_SEARCH = os.path.normpath(FILE_INPUT_SEARCH)

print(f"Search file: {FILE_INPUT_SEARCH}")

logging.info(f"Search file in: {FILE_INPUT_SEARCH}")
df_bibliometrix = load_csv_to_dataframe(FILE_INPUT_SEARCH, '|')

if df_bibliometrix is None:
    print("Erro: 'df_bibliometrix' está definido como None.")
else:
    print(f"'df_bibliometrix' contém {len(df_bibliometrix)} registros.")

pd.set_option('display.max_columns', None)
df_bibliometrix.head(2)

Search file: c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed\metadata\Bibliometrix_Metadata.csv
'df_bibliometrix' contém 14150 registros.


,PMID,ArticleTitle,JournalTitle,ISOAbbreviation,Country,Volume,Pages,ISSN,Language,AbstractText,CopyrightInformation,DOI,PII,PublicationYear,PublicationSeason,Authors,Affiliations,Keywords,MeshTerms,ChemicalSubstances,GrantIDs,GrantOrganizations,DocumentTypes,Status,LastRevisionDate,CompletionDate,PublicationHistory,ConflictOfInterest,PMCID,Citations
0,36444971,Practically Defined Off-State Dyskinesia Follo...,Movement disorders : official journal of the M...,Mov Disord,United States,38,104-112,1531-8257,eng,BACKGROUND: We recently showed that by employi...,© 2022 The Authors. Movement Disorders publish...,10.1002/mds.29262,NaN,2023,NaN,"Lloyd, Katherine K; Lawton, Michael M; Whone, ...","Translational Health Sciences, Bristol Medical...",NaN,Humans; Dopamine; Dyskinesias; Glial Cell Line...,Dopamine; fluorodopa F 18; Glial Cell Line-Der...,NaN,NaN,"Journal Article; Research Support, Non-U.S. Gov't",MEDLINE,2023-02-22,2023-01-25,revised: 2022-9-9; received: 2022-5-15; accept...,NaN,NaN,"Parmar M, Grealish S, Henchcliffe C. The futur..."
1,36503211,Regulation of Αlpha-Synuclein Gene (SNCA) by E...,International neurourology journal,Int Neurourol J,Korea (South),26,S85-93,2093-4777,eng,PURPOSE: Deregulation of SNCA encoding α-synuc...,NaN,10.5213/inj.2222206.103,NaN,2022,NaN,"Guhathakurta, Subhrangshu S; Song, Min Kyung M...","Division of Neurosciences, Burnett School of B...",Parkinson disease; Repressor; TET1; Αlpha-synu...,NaN,NaN,5R21NS088923-02;,Korea National Institute of Health; Michael J ...,Journal Article,PubMed-not-MEDLINE,2023-01-06,0000-00-00,received: 2022-9-15; accepted: 2022-10-12; ent...,,PMC9767688,"Xu L, Pu J. Alpha-synuclein in Parkinson’s dis..."


In [43]:
import pandas as pd

# Copiar o DataFrame de bibliometrix
df_search = df_bibliometrix.copy()

# Remover valores NaN na coluna 'DOI' e 'ArticleTitle'
df_search_clean = df_search.dropna(subset=['DOI', 'ArticleTitle'])

# Verificar duplicatas na coluna 'DOI'
duplicated_dois = df_search_clean[df_search_clean['DOI'].duplicated(keep=False)]

# DataFrame com DOIs duplicados
df_duplicated_dois = duplicated_dois[['DOI']].drop_duplicates()
df_duplicated_dois['count'] = (
    duplicated_dois['DOI']
    .value_counts()
    .reindex(df_duplicated_dois['DOI'])
    .values
)

# Verificar duplicatas na coluna 'ArticleTitle'
duplicated_titles = df_search_clean[df_search_clean['ArticleTitle'].duplicated(keep=False)]

# DataFrame com títulos duplicados
df_duplicated_titles = duplicated_titles[['ArticleTitle']].drop_duplicates()
df_duplicated_titles['count'] = (
    duplicated_titles['ArticleTitle']
    .value_counts()
    .reindex(df_duplicated_titles['ArticleTitle'])
    .values
)

# Verificar duplicatas na combinação 'ArticleTitle' + 'DOI'
duplicated_combinations = df_search_clean[df_search_clean.duplicated(subset=['ArticleTitle', 'DOI'], keep=False)]

# DataFrame com combinações duplicadas
df_duplicated_combinations = (
    duplicated_combinations[['ArticleTitle', 'DOI']]
    .value_counts()
    .reset_index()
    .rename(columns={0: 'count'})
)

# Exibir os resultados no console
print("DOIs Duplicados:")
print(df_duplicated_dois)

print("\nTítulos Duplicados:")
print(df_duplicated_titles)

print("\nCombinações Duplicadas (Título + DOI):")
print(df_duplicated_combinations)

# (Opcional) Salvar os resultados em arquivos CSV
df_duplicated_dois.to_csv("duplicated_dois_pot_process.csv", index=False)
df_duplicated_titles.to_csv("duplicated_titles_pot_process.csv", index=False)
df_duplicated_combinations.to_csv("duplicated_combinations_pot_process.csv", index=False)


DOIs Duplicados:
                                 DOI  count
0                  10.1002/mds.29262      2
1            10.5213/inj.2222206.103      2
2                  10.1002/mds.29290      2
3         10.1007/s10072-022-06516-8      2
4                   10.1002/sim.9604      2
...                              ...    ...
11200         10.1186/1750-1326-3-19      2
11202              10.1159/000113697      2
11205     10.2174/157015907782793667      2
11226         10.1002/prca.200700081      2
11253  10.1016/S0072-9752(07)83006-5      2

[6317 rows x 2 columns]

Títulos Duplicados:
                                            ArticleTitle  count
0      Practically Defined Off-State Dyskinesia Follo...      2
1      Regulation of Αlpha-Synuclein Gene (SNCA) by E...      2
2      Implications of DNA Methylation in Lewy Body P...      2
3      Movement disorders in hereditary spastic parap...      2
4      Spatially informed Bayesian neural network for...      2
...                      

In [8]:
# Parsing metadata to format df_bibliometrix
FILE_PARSING_BIBLIOMETRIX = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_bibliometrix", "Bibliometrix.csv"))
FILE_PARSING_BIBLIOMETRIX = os.path.normpath(FILE_PARSING_BIBLIOMETRIX)

In [13]:
# Parsing metadata to format df_bibliometrix
FILE_PARSING_BIBLIOMETRIX = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_bibliometrix", "Bibliometrix.csv"))
FILE_PARSING_BIBLIOMETRIX = os.path.normpath(FILE_PARSING_BIBLIOMETRIX)

sample_columns = ['AU', 'AF', 'CR', 'AB', 'C1', 'DE', 'DI', 'DT', 'FU', 'FX', 
                  'ID', 'IS', 'J9', 'JI', 'LA', 'OA', 'PU', 'PY', 'RP', 'SO', 
                  'TC', 'TI', 'UT', 'VL', 'C1raw', 'DB', 'AU_UN', 'AU1_UN', 
                  'AU_UN_NR', 'SR_FULL', 'SR']

#map_pubmed_to_bibliometrix(df_bibliometri,FILE_PARSING_BIBLIOMETRIX)
map_pubmed_to_bibliometrix_extended(df_bibliometrix, FILE_PARSING_BIBLIOMETRIX, sample_columns)

#df_bibliometrix.to_excel(FILE_PARSING_BIBLIOMETRIX, index=False)

logging.info(f"Saving Bibliometrix file: {FILE_PARSING_BIBLIOMETRIX}")    


c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\scripts\mapping.py:559: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(clean_surrogates)


Arquivo processado salvo em: c:\Users\ceagre\OneDrive\Projetos Private\Doutorado - Engenharia Biomédica\4_Tese\3_CodigoFonte\12_SoftwareAndImpacts\data\processed\files_type\Bibliometrix_format.xlsx


In [16]:
data = df_bibliometrix.copy()
data.fillna("", inplace=True)
data

,PMID,ArticleTitle,JournalTitle,ISOAbbreviation,Country,Volume,Pages,ISSN,Language,AbstractText,CopyrightInformation,DOI,PII,PublicationYear,PublicationSeason,Authors,Affiliations,Keywords,MeshTerms,ChemicalSubstances,GrantIDs,GrantOrganizations,DocumentTypes,Status,LastRevisionDate,CompletionDate,PublicationHistory,ConflictOfInterest,PMCID,Citations
0,36444971,Practically Defined Off-State Dyskinesia Follo...,Movement disorders : official journal of the M...,Mov Disord,United States,38,104-112,1531-8257,eng,BACKGROUND: We recently showed that by employi...,© 2022 The Authors. Movement Disorders publish...,10.1002/mds.29262,,2023,,"Lloyd, Katherine K; Lawton, Michael M; Whone, ...","Translational Health Sciences, Bristol Medical...",,Humans; Dopamine; Dyskinesias; Glial Cell Line...,Dopamine; fluorodopa F 18; Glial Cell Line-Der...,,,"Journal Article; Research Support, Non-U.S. Gov't",MEDLINE,2023-02-22,2023-01-25,revised: 2022-9-9; received: 2022-5-15; accept...,,,"Parmar M, Grealish S, Henchcliffe C. The futur..."
1,36503211,Regulation of Αlpha-Synuclein Gene (SNCA) by E...,International neurourology journal,Int Neurourol J,Korea (South),26,S85-93,2093-4777,eng,PURPOSE: Deregulation of SNCA encoding α-synuc...,,10.5213/inj.2222206.103,,2022,,"Guhathakurta, Subhrangshu S; Song, Min Kyung M...","Division of Neurosciences, Burnett School of B...",Parkinson disease; Repressor; TET1; Αlpha-synu...,,,5R21NS088923-02;,Korea National Institute of Health; Michael J ...,Journal Article,PubMed-not-MEDLINE,2023-01-06,0000-00-00,received: 2022-9-15; accepted: 2022-10-12; ent...,,PMC9767688,"Xu L, Pu J. Alpha-synuclein in Parkinson’s dis..."
2,36444149,Implications of DNA Methylation in Lewy Body P...,Movement disorders : official journal of the M...,Mov Disord,United States,38,25,1531-8257,eng,,,10.1002/mds.29290,,2023,,"Adebisi, Benjamin B; Nour, Mohamed A MA; Periñ...","Department of Anatomy, College of Health Scien...",,Humans; Lewy Bodies;pathology; DNA Methylation...,alpha-Synuclein,,,"Journal Article; Research Support, Non-U.S. Go...",MEDLINE,2023-02-22,2023-01-20,revised: 2022-11-9; received: 2022-11-1; accep...,,,"Chia R, Sabir MS, Bandres-Ciga S, et al. Genom..."
3,36441344,Movement disorders in hereditary spastic parap...,Neurological sciences : official journal of th...,Neurol Sci,Italy,44,947-959,1590-3478,eng,BACKGROUND: Hereditary spastic paraplegia (HSP...,© 2022. The Author(s).,10.1007/s10072-022-06516-8,,2023,,"Fereshtehnejad, Seyed-Mohammad SM; Saleh, Phil...","Division of Neurology, Department of Medicine,...",Genotype; Hereditary spastic paraplegia (HSP);...,"Humans; Spastic Paraplegia, Hereditary; Parapl...","SPG11 protein, human; Proteins",,,Systematic Review; Meta-Analysis; Journal Article,MEDLINE,2024-01-12,2023-02-15,received: 2022-9-19; accepted: 2022-11-17; pub...,"In the past year, S. M. F. has received honora...",PMC9925593,Harding AE. Classification of the hereditary a...
4,36440818,Spatially informed Bayesian neural network for...,Statistics in medicine,Stat Med,England,42,105-121,1097-0258,eng,Magnetic resonance imaging (MRI) plays an incr...,© 2022 The Authors. Statistics in Medicine pub...,10.1002/sim.9604,,2023,,"Payares-Garcia, David D; Mateu, Jorge J; Schic...",ITC Faculty Geo-Information Science and Earth ...,Bayesian inference; classification; deep learn...,Humans; Neurodegenerative Diseases;diagnostic ...,,U01 AG024904,NIA NIH HHS,"Journal Article; Research Support, N.I.H., Ext...",MEDLINE,2023-02-08,2022-12-27,revised: 2022-9-20; received: 2021-7-22; accep...,,,"Dugger BN, Dickson DW. Pathology of neurodegen..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14145,15210960,The role of iron and copper molecules in the n...,Proceedings of the National Academy of Science...,Proc Natl Acad Sci U S A,United States,101,9843-8,0027-8424,eng,"In this study, a comparative analysis of metal...",,,,2004,,"Zecca, Luigi L; Stroppolo, 

In [31]:
import pandas as pd
import re

 # Lista de campos extraídos para análise bibliométrica
    # Cada campo representa uma informação específica da base de dados utilizada (e.g., Web of Science, Scopus):
    # AU: Nome(s) do(s) autor(es) do artigo.
    # AF: Nome completo do(s) autor(es), conforme aparece na publicação.
    # CR: Referências citadas no artigo (citações listadas no estilo usado na publicação).
    # AB: Resumo do artigo, descrevendo os principais objetivos e descobertas.
    # C1: Endereços institucionais afiliados ao(s) autor(es), com nomes de instituições e departamentos.
    # DE: Palavras-chave do autor, indicando os principais tópicos abordados no artigo.
    # DI: DOI (Digital Object Identifier), identificador digital único para o artigo.
    # DT: Tipo de documento (e.g., artigo, revisão, conferência).
    # FU: Fontes de financiamento ou agências de fomento que apoiaram o trabalho.
    # FX: Informações adicionais fornecidas no campo de agradecimentos.
    # ID: Palavras-chave atribuídas por indexadores (diferentes das palavras-chave do autor).
    # IS: Número da edição do periódico onde o artigo foi publicado.
    # J9: Nome abreviado do periódico (abreviação padrão).
    # JI: ISSN do periódico (International Standard Serial Number).
    # LA: Idioma em que o artigo foi publicado.
    # OA: Indica se o artigo é de acesso aberto (open access).
    # PU: Editora responsável pela publicação do periódico ou conferência.
    # PY: Ano de publicação do artigo.
    # RP: Endereço de correspondência do autor principal.
    # SO: Nome completo do periódico onde o artigo foi publicado.
    # TC: Número total de citações recebidas pelo artigo.
    # TI: Título do artigo.
    # UT: Número de registro único atribuído pela base de dados (e.g., Web of Science).
    # VL: Volume do periódico onde o artigo foi publicado.
    # C1raw: Campo bruto contendo informações não processadas sobre endereços institucionais (antes da normalização).
    # DB: Base de dados de onde os dados bibliométricos foram extraídos (e.g., Web of Science, Scopus).
    # AU_UN: Instituições afiliadas ao(s) autor(es), após normalização.
    # AU1_UN: Instituição afiliada ao primeiro autor, após normalização.
    # AU_UN_NR: Número de instituições únicas afiliadas ao(s) autor(es).
    # SR_FULL: Referência completa (Source Reference), incluindo título, autor(es), periódico e outros detalhes da publicação.
    # SR: Referência abreviada (citação simplificada usada na análise).

order_column = ['AU', 'AF', 'CR', 'AB', 'C1', 'DE', 'DI', 'DT', 'FU', 'FX', 
                  'ID', 'IS', 'J9', 'JI', 'LA', 'OA', 'PU', 'PY', 'RP', 'SO', 
                  'TC', 'TI', 'UT', 'VL', 'C1raw', 'DB', 'AU_UN', 'AU1_UN', 
                  'AU_UN_NR', 'SR_FULL', 'SR']


mapping = {
    "Authors": "AU",
    "Citations": "CR",
    "AbstractText": "AB",
    "Affiliations": "C1",
    "Keywords": "DE",
    "DOI": "DI",
    "DocumentTypes": "DT",
    "GrantOrganizations": "FU",
    "ChemicalSubstances": "ID",
    "ISOAbbreviation": "J9",
    "ISSN": "JI",
    "Language": "LA",
    "PublicationYear": "PY",
    "JournalTitle": "SO",
    "ArticleTitle": "TI",
    "PMID": "UT",
    "Volume": "VL",
    "PUBMED": "DB",
    "Country": "SO_CO",
    "MeshTerms": "MESH",
    "ISSN": "SN",
    "Pages": "PG",
    "GrantIDs": "GRANT_ID",
}

# Functions for cleaning and processing data
def clean_affiliation(affiliation):
    if not affiliation:
        return ""
    affiliation = re.sub(r"\[.*?\]", "", affiliation)
    return re.sub(r"\s+", " ", affiliation.strip())

# Função para formatar nomes no formato esperado
def format_authors(authors):
    if pd.notna(authors):
        authors_list = authors.split(";")  # Separar autores
        formatted_authors = []
        for author in authors_list:
            parts = author.strip().split(",")  # Separar sobrenome do resto
            if len(parts) == 2:
                surname = parts[0].strip().upper()  # Sobrenome em maiúsculas
                first_initial = parts[1].strip()[0].upper()  # Apenas a primeira inicial
                formatted_authors.append(f"{surname} {first_initial}")
        return ";".join(formatted_authors)
    return ""

# Função para formatar os autores no formato especificado
def format_full_authors(authors):
    if pd.notna(authors):
        authors_list = authors.split(";")  # Separar autores por ponto e vírgula
        formatted_authors = []
        for author in authors_list:
            parts = author.strip().split(",")  # Dividir sobrenome e restante do nome
            if len(parts) == 2:
                surname = parts[0].strip().upper()  # Sobrenome em maiúsculas
                names = parts[1].strip().split()  # Nome completo e possíveis iniciais
                # Combinar nomes intermediários e adicionar ponto às iniciais
                first_name = " ".join(
                    [name.upper() + "." if len(name) == 1 else name.upper() for name in names[:-1]]
                )  # Nomes intermediários com ponto nas iniciais
                last_initial = f"{names[-1].upper()}." if len(names[-1]) == 1 else names[-1].upper()  # Última inicial com ponto
                cleaned_author = f"{surname}, {first_name} {last_initial}".strip()
                formatted_authors.append(cleaned_author)

        # Retornar todos os autores formatados, incluindo os últimos
        return ";".join(formatted_authors)
    return ""


# Dados de exemplo com valores a serem padronizados
data = {
    "DT": [
        "ARTICLE",
        "REVIEW",
        "ARTICLE; EARLY ACCESS",
        "PROCEEDINGS PAPER",
        "ARTICLE; PROCEEDINGS PAPER",
        "MEETING ABSTRACT",
        "LETTER",
        "EDITORIAL MATERIAL",
        "CORRECTION",
        "LETTER; EARLY ACCESS",
        "CONFERENCE PAPER",
        "ERRATUM",
        "NOTE",
        "BOOK CHAPTER"
    ]
}


# Função para realizar o mapeamento
def map_values(value):
    # Caso o valor exista no dicionário, retorna o mapeado; caso contrário, mantém o original
    return mapping.get(value, value)

def generate_short_reference(row):
    first_author = row["Authors"].split(";")[0].strip() if "Authors" in row and pd.notna(row["Authors"]) else "NA"
    #year = str(int(row["PublicationYear"])) if "PublicationYear" in row and pd.notna(row["PublicationYear"]) else "NA"
    year = (
        str(int(row["PublicationYear"])) 
        if "PublicationYear" in row and pd.notna(row["PublicationYear"]) and str(row["PublicationYear"]).strip().isdigit()
        else "NA"
    )
    #year = str(int(row["PublicationYear"])) if "PublicationYear" in row and pd.notna(row["PublicationYear"]) else "NA"
    source = row["JournalTitle"] if "JournalTitle" in row and pd.notna(row["JournalTitle"]) else "NA"
    return f"{first_author}, {year}, {source}"

def clean_surrogates(text):
    if isinstance(text, str):
        return text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    return text

# Normalize data
data = df_bibliometrix.copy()
df_mapped = data.rename(columns=mapping)
#df_mapped

df_mapped.fillna("", inplace=True)

# Remove invalid characters
df_mapped = df_mapped.applymap(clean_surrogates)


data["SR_FULL"] = data.apply(generate_short_reference, axis=1)
data["SR"] = data["SR_FULL"]

if "AU" in df_mapped.columns:
    df_mapped["AF"] = df_mapped["AU"]
    df_mapped["AF"] = df_mapped["AF"].apply(format_full_authors)

    df_mapped["AU"] = df_mapped["AU"].apply(format_authors)

if "FU" in df_mapped.columns:
    df_mapped["GRANT_ORG"] = df_mapped["FU"]

if "VL" in df_mapped.columns:
    df_mapped["IS"] = df_mapped["VL"]

if "DT" in df_mapped.columns:
    df_mapped["DT"] = df_mapped["DT"].apply(map_values)

df_mapped["FX"] = ""
df_mapped["OA"] = ""
df_mapped["PU"] = ""
df_mapped["RP"] = ""
df_mapped["TC"] = ""


# Verifica quais colunas de 'order_column' estão presentes em 'df_mapped'
ordered_columns = [col for col in order_column if col in df_mapped.columns]

# Reorganiza as colunas no DataFrame
df_ordered = df_mapped[ordered_columns]
df_ordered

#df_ordered.to_excel(FILE_PARSING_BIBLIOMETRIX, index=False, engine='openpyxl')
#print(f"Arquivo processado salvo em: {FILE_PARSING_BIBLIOMETRIX}")


C:\Users\ceagre\AppData\Local\Temp\ipykernel_17216\1406128667.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_mapped = df_mapped.applymap(clean_surrogates)


,AU,AF,CR,AB,C1,DE,DI,DT,FU,FX,ID,IS,J9,LA,OA,PU,PY,RP,SO,TC,TI,UT,VL
0,LLOYD K;LAWTON M;WHONE A,"LLOYD, KATHERINE K.;LAWTON, MICHAEL M.;WHONE, ...","Parmar M, Grealish S, Henchcliffe C. The futur...",BACKGROUND: We recently showed that by employi...,"Translational Health Sciences, Bristol Medical...",,10.1002/mds.29262,"Journal Article; Research Support, Non-U.S. Gov't",,,Dopamine; fluorodopa F 18; Glial Cell Line-Der...,38,Mov Disord,eng,,,2023,,Movement disorders : official journal of the M...,,Practically Defined Off-State Dyskinesia Follo...,36444971,38
1,GUHATHAKURTA S;SONG M;BASU S;JE G;CRISTOVAO A;...,"GUHATHAKURTA, SUBHRANGSHU S.;SONG, MIN KYUNG M...","Xu L, Pu J. Alpha-synuclein in Parkinson’s dis...",PURPOSE: Deregulation of SNCA encoding α-synuc...,"Division of Neurosciences, Burnett School of B...",Parkinson disease; Repressor; TET1; Αlpha-synu...,10.5213/inj.2222206.103,Journal Article,Korea National Institute of Health; Michael J ...,,,26,Int Neurourol J,eng,,,2022,,International neurourology journal,,Regulation of Αlpha-Synuclein Gene (SNCA) by E...,36503211,26
2,ADEBISI B;NOUR M;PERIÑÁN M,"ADEBISI, BENJAMIN B.;NOUR, MOHAMED A. MA;PERIÑ...","Chia R, Sabir MS, Bandres-Ciga S, et al. Genom...",,"Department of Anatomy, College of Health Scien...",,10.1002/mds.29290,"Journal Article; Research Support, Non-U.S. Go...",,,alpha-Synuclein,38,Mov Disord,eng,,,2023,,Movement disorders : official journal of the M...,,Implications of DNA Methylation in Lewy Body P...,36444149,38
3,FERESHTEHNEJAD S;SALEH P;OLIVEIRA L;PATEL N;BH...,"FERESHTEHNEJAD, SEYED-MOHAMMAD SM;SALEH, PHILI...",Harding AE. Classification of the hereditary a...,BACKGROUND: Hereditary spastic paraplegia (HSP...,"Division of Neurology, Department of Medicine,...",Genotype; Hereditary spastic paraplegia (HSP);...,10.1007/s10072-022-06516-8,Systematic Review; Meta-Analysis; Journal Article,,,"SPG11 protein, human; Proteins",44,Neurol Sci,eng,,,2023,,Neurological sciences : official journal of th...,,Movement disorders in hereditary spastic parap...,36441344,44
4,PAYARES-GARCIA D;MATEU J;SCHICK W,"PAYARES-GARCIA, DAVID D.;MATEU, JORGE J.;SCHIC...","Dugger BN, Dickson DW. Pathology of neurodegen...",Magnetic resonance imaging (MRI) plays an incr...,ITC Faculty Geo-Information Science and Earth ...,Bayesian inference; classification; deep learn...,10.1002/sim.9604,"Journal Article; Research Support, N.I.H., Ext...",NIA NIH HHS,,,42,Stat Med,eng,,,2023,,Statistics in medicine,,Spatially informed Bayesian neural network for...,36440818,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14145,ZECCA L;STROPPOLO A;GATTI A;TAMPELLINI D;TOSCA...,"ZECCA, LUIGI L.;STROPPOLO, ANTONELLA A.;GATTI,...","Foote, S. L., Freedman, R. & Oliver, A. P. (19...","In this study, a comparative analysis of metal...","Institute of Biomedical Technologies, Italian ...",,,"Journal Article; Research Support, Non-U.S. Go...",,,Iron Chelating Agents; Melanins; neuromelanin;...,101,Proc Natl Acad Sci U S A,eng,,,2004,,Proceedings of the National Academy of Science...,,The role of iron and copper molecules in the n...,15210960,101
14146,LOCKHART P;O'FARRELL C;FARRER M,"LOCKHART, PAUL J. PJ;O'FARRELL, CASEY A. CA;FA...",,Mutations in the parkin gene (PRKN) are the co...,"Mayo Clinic Jacksonville, Jacksonville, Florid...",,,"Journal Article; Research Support, Non-U.S. Go...",,,Microfilament Proteins; Molecular Chaperones; ...,19,Mov Disord,eng,,,2004,,Movement disorders : official journal of the M...,,It's a double knock-out! The quaking mouse is ...,14743368,19
14147,CASTELO-BRANCO G;WAGNER J;RODRIGUEZ F;KELE J;S...,"CASTELO-BRANCO, GONÇALO G.;WAGNER, JOSEPH J.;R...","Zetterstrom, R. H., Solomin, L., Jansson, L., ...",The Wnts are a family of glycoproteins that re...,"Laboratory of Molecular Neurobiology, Medical ...",,,"Journal Article; Research Support, Non-U.S. Go...",NCI NIH HHS,,"CTNNB1 protein, mouse; Ctnnb1 protein, rat; Cy...",100,Proc Nat

In [9]:
FILE_PARSING_VSVIWER = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_vsviewer", "file_vsviewer.ris"))

# Convert the dataframe to RIS format
ris_data = map_pubmed_to_ris(df_bibliometrix)

# Save the RIS data to a file
with open(FILE_PARSING_VSVIWER, 'w') as f:
    f.write(ris_data)

logging.info(f"Saving file VSViewer file: {FILE_PARSING_VSVIWER}")    
print(f"Saving file VSViewer: {FILE_PARSING_VSVIWER}")


KeyError: 'AU'

In [ ]:
# Convert the dataframe to BibTeX format
BIBTEX_FILE_PATH = os.path.join(OUTPUT_PATH, CONFIG["files"].get("file_bibtex", "bibtex.bib"))
bibtex_data = map_pubmed_to_bibtex(df_bibliometrix)

# Save the BibTeX data to a file
with open(BIBTEX_FILE_PATH, 'w') as f:
    f.write(bibtex_data)

logging.info(f"Saving file VSViewer file: {BIBTEX_FILE_PATH}")    
print(f"Saving file VSViewer: {BIBTEX_FILE_PATH}")
